In [1]:
import pixiedust
pixiedust.enableJobMonitor()

Pixiedust database opened successfully


Succesfully enabled Spark Job Progress Monitor


In [2]:
"""
Extract head and content from file
"""
rdd = sc.textFile('../Project/Data/Broilers_Data.txt')

lineSpliter = rdd.map(lambda x: x.split('\t'))
print("The file has {0} lines\n".format(lineSpliter.count()))

oldHeader = ['MainKey']

header = ['Main_Key', 'Date', 'Customer_Code', 'House', 'Flock', 'Gene_Line', 'Birds_Begin', 'Hatch_Date', 'Arrive_Date',\
 'Remove_date', 'Deactivate_Date', 'Veterinarian', 'Hatchery', 'Age', 'Birds_Present', 'Birds_thinned', 'Death',\
 'Death_Cum', 'Total_Death_Rate', 'Alive_Rate', 'Body_Weight_g', 'Uniformity_Rate', 'Daily_Gaing', 'Avg_Daily_Gaing_Per_Day',\
 'Feed_Intake_Per_house_kg', 'FCR_Cum', 'Wheat_Per_Bird_Cum', 'Wheat_Per_Bird', 'Wheat_Per_Day',\
 'Feed_Intake_Per_Bird_Housed_Cum_kg', 'Feed_Intake_Per_Bird_g', 'Wheat_g', 'FCR', 'Water_l', 'Water_Intake_Per_Bird_ml',\
 'Water_Intake_Per_Bird_Cum_l', 'Water_Feed', 'Water_FeedCum', 'Comment']

# Codes in for-loop is useless now
for col in lineSpliter.take(1)[0]:
    oldHeader.append(col.strip()
                     .replace(' ', '')
                     .replace('#', '')
                     .replace('(', '')
                     .replace(')', '')
                     .replace('.', '')
                     .replace('/', 'Per')
                     .replace('%', 'Percentage'))

print("The header of this file is \n{0}\n".format(header))
print("The header of this file has {0} lines".format(len(header)))

data = lineSpliter.filter(lambda x: x[0] != "Date")
#print("Data in this file has {0} lines".format(data.count()))

def lengthInspector(rdd):
    lenCounter = rdd.map(lambda x: (len(x), 1))\
                 .reduceByKey(lambda x, y: x + y)
    lenList = lenCounter.collect()
    return lenList

lenList = lengthInspector(data)
print("Distribution of length of lines in this file: ")
print(lenList)

"""
Clean the empty lines, calculate main key and clean illegal main key
"""
def buildMainKey(line):
    mainKey = "Farm " + line[1].strip() + " House " + line[2].strip() + " Flock " + line[3].strip()
    newline = [mainKey] + line
    return newline

emptyClener = data.filter(lambda x: len(x) >= 38)
#print("Not empty data has {0} lines".format(emptyClener.count()))

mainKeyCounter = emptyClener.map(lambda x: buildMainKey(x))\
                            .map(lambda x: (x[0], 1))\
                            .reduceByKey(lambda x, y: x + y)
print("\nThere are {0} different main keys".format(mainKeyCounter.count()))

errorMainKeyCleaner = emptyClener.filter(lambda x: False if x[3] == "" else True)
print("There are {0} lines with vaild main key".format(errorMainKeyCleaner.count()))

cleanData = errorMainKeyCleaner.map(lambda x: buildMainKey(x))
cleanMainKeyCounter = cleanData.map(lambda x: (x[0], 1))\
                               .reduceByKey(lambda x, y: x + y)
mainKeysWithCounts = cleanMainKeyCounter.collect()
mainKeys = cleanMainKeyCounter.map(lambda x: x[0]).collect()
#print("The main keys are:")
for mainkey in mainKeys:
    pass#print(mainkey)

The file has 3677 lines

The header of this file is 
['Main_Key', 'Date', 'Customer_Code', 'House', 'Flock', 'Gene_Line', 'Birds_Begin', 'Hatch_Date', 'Arrive_Date', 'Remove_date', 'Deactivate_Date', 'Veterinarian', 'Hatchery', 'Age', 'Birds_Present', 'Birds_thinned', 'Death', 'Death_Cum', 'Total_Death_Rate', 'Alive_Rate', 'Body_Weight_g', 'Uniformity_Rate', 'Daily_Gaing', 'Avg_Daily_Gaing_Per_Day', 'Feed_Intake_Per_house_kg', 'FCR_Cum', 'Wheat_Per_Bird_Cum', 'Wheat_Per_Bird', 'Wheat_Per_Day', 'Feed_Intake_Per_Bird_Housed_Cum_kg', 'Feed_Intake_Per_Bird_g', 'Wheat_g', 'FCR', 'Water_l', 'Water_Intake_Per_Bird_ml', 'Water_Intake_Per_Bird_Cum_l', 'Water_Feed', 'Water_FeedCum', 'Comment']

The header of this file has 39 lines
Distribution of length of lines in this file: 
[(38, 3676)]

There are 76 different main keys
There are 3673 lines with vaild main key


In [3]:
"""

"""
mainKeyArror = range(1, 74)
numMainKey = dict(zip(mainKeyArror, mainKeys))
print(numMainKey)

dataIndexArror = range(39)
numDataIndex = dict(zip(dataIndexArror, header))
print(numDataIndex)

def dataTaker(mainKey, dataIndex):
    """
    Return a list of data under given main key and data idx
    """
    if type(mainKey) is int:
        mainKey = numMainKey[mainKey]
    if type(dataIndex) is str:
        for key in numDataIndex:
            if dataIndex == numDataIndex[key]: 
                dataIndex = key
                break
    
    result = cleanData.filter(lambda line: line[0] == mainKey)\
                      .map(lambda line: line[dataIndex])\
                      .collect() # dataIndex should not minus 1 because main key is line[0]
    return result

{1: 'Farm B House 3 Flock Nov-16', 2: 'Farm B House 2 Flock Nov-16', 3: 'Farm B House 1 Flock Nov-16', 4: 'Farm B House 2 Flock Jan-17', 5: 'Farm B House 1 Flock Jan-17', 6: 'Farm C House 2 Flock Sep-16', 7: 'Farm C House 2 Flock Koppel 11 stal 2', 8: 'Farm D House 1 Flock Jul-16', 9: 'Farm D House 2 Flock Jul-16', 10: 'Farm D House 3 Flock Sep-16', 11: 'Farm D House 1 Flock Nov-16', 12: 'Farm D House 3 Flock Nov-16', 13: 'Farm D House 2 Flock Nov-16', 14: 'Farm D House 2 Flock Dec-16', 15: 'Farm D House 1 Flock Dec-16', 16: 'Farm D House 1 Flock Apr-17', 17: 'Farm D House 2 Flock Apr-17', 18: 'Farm D House 2 Flock Jun-17', 19: 'Farm D House 1 Flock Jun-17', 20: 'Farm D House 3 Flock Jun-17', 21: 'Farm D House 3 Flock Aug-17', 22: 'Farm D House 2 Flock Aug-17', 23: 'Farm D House 2 Flock Oct-17', 24: 'Farm D House 1 Flock Oct-17', 25: 'Farm D House 3 Flock Oct-17', 26: 'Farm E House 7 Flock Sep-16', 27: 'Farm E House 5 Flock Sep-16', 28: 'Farm E House 6 Flock Nov-16', 29: 'Farm E House 

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/pixiedust/utils/sparkJobProgressMonitor.py", line 47, in startSparkJobProgressMonitor
    progressMonitor = SparkJobProgressMonitor()
  File "/opt/conda/lib/python3.6/site-packages/pixiedust/utils/sparkJobProgressMonitor.py", line 174, in __init__
    self.addSparkListener()
  File "/opt/conda/lib/python3.6/site-packages/pixiedust/utils/sparkJobProgressMonitor.py", line 203, in addSparkListener
    _env.getTemplate("sparkJobProgressMonitor/addSparkListener.scala").render()
  File "/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2131, in run_cell_magic
    result = fn(magic_arg_s, cell)
  File "<decorator-gen-126>", line 2, in scala
  File "/opt/

In [4]:
def reduceDot(linex, liney):
    new_list = []
    for i in range(len(linex)):
        new_list.append(linex[i] or liney[i])
    return new_list

checkAllDot = cleanData.map(lambda x: [('.' in i) for i in x])\
                       .reduce(lambda x, y: reduceDot(x, y))
print(checkAllDot)

def numberCleaner(line):
    """
    This function needs to be improved
    """
    resultList = []
    item = None
    for idx in range(len(line)):
        if idx <= 12 or idx == 38:
            if idx == 6: 
                if not line[idx]: item = -1
                else:
                    item = line[idx].strip().replace(',', '').replace(' ', '')
                    item = int(item)
            else: 
                if not line[idx]: item = ""
                else: item = line[idx]
        elif checkAllDot[idx]:
            if not line[idx]: item = -0.1
            else:
                item = line[idx].strip().replace(',', '').replace(' ', '')
                item = float(item)
        else: 
            if not line[idx]: item = -1
            else:
                item = line[idx].strip().replace(',', '').replace(' ', '').replace('"', '')
                item = int(item)
        resultList.append(item)
    return resultList

numCleanData = cleanData.map(numberCleaner)

def transferNone(line):
    new_line = []
    for item in line:
        if type(item) != str:
            if item < 0: new_line.append(None)
            else: new_line.append(item)
        else:
            if not item: new_line.append(None)
            else:new_line.append(item)
    return new_line

broilers = numCleanData.map(transferNone)

broilersDF = sqlContext.createDataFrame(numCleanData, header)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [35]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf

def selectArea(Main_Keys, inCols):
    selectRows = udf(lambda x: x in Main_Keys, BooleanType())
    area = broilersDF.filter(selectRows(broilersDF.Main_Key)).select(inCols)
    return area

print(header)
print(mainKeys)

test = ['Age', 'Death']

def selectRDD(inRows, inCols):
    """
    !!!Replace dataTaker!!!
    
    Return RDD containing given main keys and data idxs
    """
    newRows = []
    newCols = []
    for row in inRows:
        if type(row) is int: newRows.append(numMainKey[row])
        else: newRows.append(row)
    for col in inCols:
        if type(col) is str:
            for key in numDataIndex:
                if col == numDataIndex[key]: 
                    newCols.append(key)
                    break
        else: newCols.append(col)
    
    newRDD = broilers.filter(lambda line: line[0] in newRows)\
                      .map(lambda line: [line[idx] for idx in newCols])
    #result = sqlContext.createDataFrame(newRDD, inCols)
    return inCols, newRDD

testCols, testRDD = selectRDD(mainKeys[1:3], test)
trainCols, trainRDD = selectRDD(mainKeys[2:], test)

['Main_Key', 'Date', 'Customer_Code', 'House', 'Flock', 'Gene_Line', 'Birds_Begin', 'Hatch_Date', 'Arrive_Date', 'Remove_date', 'Deactivate_Date', 'Veterinarian', 'Hatchery', 'Age', 'Birds_Present', 'Birds_thinned', 'Death', 'Death_Cum', 'Total_Death_Rate', 'Alive_Rate', 'Body_Weight_g', 'Uniformity_Rate', 'Daily_Gaing', 'Avg_Daily_Gaing_Per_Day', 'Feed_Intake_Per_house_kg', 'FCR_Cum', 'Wheat_Per_Bird_Cum', 'Wheat_Per_Bird', 'Wheat_Per_Day', 'Feed_Intake_Per_Bird_Housed_Cum_kg', 'Feed_Intake_Per_Bird_g', 'Wheat_g', 'FCR', 'Water_l', 'Water_Intake_Per_Bird_ml', 'Water_Intake_Per_Bird_Cum_l', 'Water_Feed', 'Water_FeedCum', 'Comment']
['Farm B House 3 Flock Nov-16', 'Farm B House 2 Flock Nov-16', 'Farm B House 1 Flock Nov-16', 'Farm B House 2 Flock Jan-17', 'Farm B House 1 Flock Jan-17', 'Farm C House 2 Flock Sep-16', 'Farm C House 2 Flock Koppel 11 stal 2', 'Farm D House 1 Flock Jul-16', 'Farm D House 2 Flock Jul-16', 'Farm D House 3 Flock Sep-16', 'Farm D House 1 Flock Nov-16', 'Farm D 

In [54]:
# This cell is for preparing datasets used in regression
from math import sqrt, log, exp

print(testRDD.take(20))

def transformRDD(inputRDD, inCols):
    outCols = inCols + ['square', 'sqrt', 'ln', 'exp']
    trRDD = inputRDD.filter(lambda x: x[0])\
                    .map(lambda x: [x[0], x[1]] if x[1] else [x[0], 0])\
                    .map(lambda x: [x[0], x[1], x[0] ** 2, sqrt(x[0]), log(x[0]), exp(x[0])])
    outDF = sqlContext.createDataFrame(trRDD, outCols)
    return outDF

testDF = transformRDD(testRDD, testCols)
trainDF = transformRDD(trainRDD, trainCols)
#trl = ['Age', 'Death', (math.log(sample_test.Death)).alias('ln'), (sample_test.Death ** 0.5).alias('Death_sqrt')]
#print("attributes of the column are: {}".format(dir(sample_test.Death)))
#trDF = sample_test.select('Age', 'Death', (math.log(sample_test.Death)).alias('ln'), (sample_test.Death ** 0.5).alias('Death_sqrt'))
testDF.show()
trainDF.show()
print(trainRDD.filter(lambda x: not x[0]).filter(lambda x: x[0]).count())

[[1, 4], [1, 3], [2, 24], [2, 55], [3, 71], [3, 64], [4, 44], [4, 33], [5, 11], [5, 17], [6, 14], [6, 16], [7, 11], [7, 20], [8, 17], [8, 13], [9, 18], [9, 19], [10, 13], [10, 13]]
+---+-----+------+------------------+------------------+------------------+
|Age|Death|square|              sqrt|                ln|               exp|
+---+-----+------+------------------+------------------+------------------+
|  1|    4|     1|               1.0|               0.0| 2.718281828459045|
|  1|    3|     1|               1.0|               0.0| 2.718281828459045|
|  2|   24|     4|1.4142135623730951|0.6931471805599453|  7.38905609893065|
|  2|   55|     4|1.4142135623730951|0.6931471805599453|  7.38905609893065|
|  3|   71|     9|1.7320508075688772|1.0986122886681098|20.085536923187668|
|  3|   64|     9|1.7320508075688772|1.0986122886681098|20.085536923187668|
|  4|   44|    16|               2.0|1.3862943611198906|54.598150033144236|
|  4|   33|    16|               2.0|1.3862943611198906|54.

In [57]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression, LinearRegressionModel, RandomForestRegressor

def Do_Machine_Learning(trainingSet, testSet, xCols, yValues, regressor = LinearRegression(),
                         paramGrid = [], evalMetric = "rmse", seed = None):
    """
    Return (<object> model, <float> error_estimate, <dataframe> result)
    
    trainingSet: dataframe, used to train model
    testSet: dataframe, used to feed the model and get the result(and error estimate)
    xCols: list of strings, names of columns which used as inputs
    yValues: string, name of column that contains dependent values 
    regressor: Regression object, by default = LinearRegression()
    paramGrid: list built byParamGridBuilder, by default = empty list
    evalMetric: string, name of matrix used for evaluation, by default = "rmse"
    seed: int or None, seed for random number generator, if == None will use random numbers
    
    !!! seed is useless at that time !!!
    """
    # push estimator into pipeline
    vec = VectorAssembler(inputCols = xCols, outputCol = "features")
    regPipeline = Pipeline()
    regPipeline.setStages([vec, regressor])   
    # build evaluator
    regEval = RegressionEvaluator(predictionCol = "Predicted", labelCol = yValues, metricName = evalMetric)
    # combine estimator and evaluator to a cross validator
    crossval = CrossValidator(estimator = regPipeline, evaluator = regEval, numFolds = 3)
    # set parameters grid
    crossval.setEstimatorParamMaps(paramGrid)
    # trainning
    regModel = crossval.fit(trainingSet).bestModel
    # predicting
    predictions = regModel.transform(testSet)
    # get evaluating result
    evaluation = regEval.evaluate(predictions)
    
    return regModel, evaluation, predictions

# build regressor
lr1 = LinearRegression()
lr1.setPredictionCol("Predicted")\
   .setLabelCol("Death")

# build parameter grid
regParam = [x / 100.0 for x in range(1, 10)]
pg1 = (ParamGridBuilder()
             .addGrid(lr1.regParam, regParam)
             .build())

print(type(lr1))
print(type(pg1))

# run ML
model1, result1, predictionDF1 = Do_Machine_Learning(trainDF, testDF, ["Age", "ln"], "Death", lr1, pg1)
model2, result2, predictionDF2 = Do_Machine_Learning(trainDF, testDF, ["Age", "square"], "Death", lr1, pg1)
model3, result3, predictionDF3 = Do_Machine_Learning(trainDF, testDF, ["Age", "exp"], "Death", lr1, pg1)


# print attributions of model
"""
print("attributes of the model are: {}".format(dir(model)))
print("method list: {}".format([method for method in dir(model) if callable(getattr(model, method))]))
print(model.stages)
"""

# Print coefficients and intercept
weights1 = model1.stages[1].coefficients
ic1 = model1.stages[1].intercept
weights2 = model2.stages[1].coefficients
ic2 = model2.stages[1].intercept
weights3 = model3.stages[1].coefficients
ic3 = model3.stages[1].intercept
print(weights1, weights2, weights3)
print(ic1, ic2, ic3)
#print(list(zip(["Age"], weights1)))
#print(model.stages[1].intercept)

# print error and result
print("Mean Squared Error: {0:2.2f}, {1:2.2f}, {2:2.2f}\n".format(result1, result2, result3))
predictionDF1.show()
predictionDF2.show()
predictionDF3.show()

# print the model
#print(model.stages)

<class 'pyspark.ml.regression.LinearRegression'>
<class 'list'>
[0.156123079079,-10.1427841846] [-0.794247528825,0.00645793136974] [-0.243627194676,9.65907404226e-59]
41.81960192255575 30.255080463237025 22.564869554122772
Mean Squared Error: 13.31, 12.99, 13.62

+---+-----+------+------------------+------------------+------------------+--------------------+------------------+
|Age|Death|square|              sqrt|                ln|               exp|            features|         Predicted|
+---+-----+------+------------------+------------------+------------------+--------------------+------------------+
|  1|    4|     1|               1.0|               0.0| 2.718281828459045|           [1.0,0.0]| 41.97572500163444|
|  1|    3|     1|               1.0|               0.0| 2.718281828459045|           [1.0,0.0]| 41.97572500163444|
|  2|   24|     4|1.4142135623730951|0.6931471805599453|  7.38905609893065|[2.0,0.6931471805...| 35.10140582009498|
|  2|   55|     4|1.4142135623730951|0.6

In [9]:
print(type(lr1))
print(pg1)

<class 'pyspark.ml.regression.LinearRegression'>
[{Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.02}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.03}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.04}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.05}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.06}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'): 0.07}, {Param(parent='LinearRegression_4dc5b27433f5df6b3c15', name='regParam', doc='regularization parameter (>= 0).'